# KNN from Scratch

Using the KNN algorithm from scikit-learn is quite simple. When you use it, you don't even need to know how KNN works and classifies the data points. The goal should be to really understand what you are doing. This is the only way to make sure you understand when something doesn't go the way you expect or want it to. Many people remember things best when they implement them themselves. So, let's build our own K-Nearest-Neighbors classifier!

**The purpose of this notebook is to help you remember the steps necessary to classify samples with KNN.**

To test if your code works, you can use the Iris dataset as a data example.
Let's make a plan and break this big task into smaller steps!


1. What information and data does the algorithm need to train and predict the classes of new instances?
This will be the input for our function! 

2. calculate the distance between the test point and each existing data point in the training data.
3. determine the nearest k neighbors.
4. make predictions based on these neighbors.

You have already implemented a function to calculate the distance between points, which will now come in handy.

A good way to get started, is to ignore the syntax and just write in simple text what you want your program to do aka **write pseudocode**. You can then start to build out some of the structure. What variables are you going to need? What kinds of logic? 
Knowing where you’re going can help you make fewer mistakes as you’re trying to get there.

Note that for large data sets, the algorithm can take very long to classify because it has to calculate the distance between the test point and every other point in the data!

You can check if your pseudo-code contains all necessary steps afterwards, when scrolling down to "KNN algorithm from scratch" where you find an example of a knn pseudocode.

## Import and Setup

In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial import distance
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Load data
df = pd.read_csv('data/iris.csv')

In [ ]:
# Defining X and y 
y = df.species
X = df.drop(['species'], axis=1)

In [ ]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)

## Distance Metrics

As already explained, KNN assigns a class to the test point based on the majority class of  K  nearest neighbors. In general, euclidean distance is used to find nearest neighbors, but other distance metrics can also be used.

As the dimensionality of the feature space increases, the euclidean distance often becomes problematic due to the curse of dimensionality (discussed later).

In such cases, alternative vector-based similarity measures (dot product, cosine similarity, etc) are used to find the nearest neighbors. This transforms the original metric space into one more amicable to point-to-point measurements.

Another distance measure that you might consider is [Mahalanobis distance](https://en.wikipedia.org/wiki/Mahalanobis_distance). Mahalanobis distance attempts to weight features according to their probabilities. On some data sets that may be important.

In general, it's probably a good idea to normalize the data at a minimum. Here's a link to the scikit-learn scaling package: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html . You have to be a little circumspect about employing any technique where the answers change with scaling.

In [ ]:
# Implement distance function on your own
def distance_self(pt1, pt2, c=2):
    dist = np.power(sum([np.abs(xi-yi)**c for xi, yi in zip(pt1, pt2)]),1/c)
    return dist

In [ ]:
# Checking if distance function works as expected. Therefore the data has to be a numpy array not a pandas dataframe. 
X_train_array = np.array(X_train)
X_test_array = np.array(X_test)

# Calculate distance with self implemented function 
print('Self implemented:', distance_self(X_train_array[0], X_test_array[0]))

# Calculating euclidean distance with numpy (as reference)
print('Numpy:', np.linalg.norm(X_train_array[0] - X_test_array[0]))

# Calculating euclidean distance with scipy (as reference)
print('Scipy:', distance.euclidean(X_train_array[0], X_test_array[0]))

## KNN ALgorithm from scratch


Remember the steps:

1. What information and data does the algorithm need to train and predict the classes of new instances?
This will be the input for our function! 

2. calculate the distance between the test point and each existing data point in the training data.
3. determine the nearest k neighbors.
4. make predictions based on these neighbors.

Hopefully you have already thought of your gameplan, also called pseudocode. If so, you can compare it to this one:
```
INPUT: X_train, y_train, X_test, k
FOR each object_to_predict in X_test:
    FOR each training_point, index in X_train:
        calculate distance d between object_to_predict and training_point
        store d and index
    SORT distances d in increasing order
    take first k items, get indices of those
    calculate most common class of points at indices in y_train (prediction)
    store prediction
RETURN list of predictions
````

Time to code!
Don't forget that it's good practice to document your own code! This way you can later understand what the purpose of each step was.
Maybe you can even use your pseudo code as documentation :)

In [ ]:
# KNN implementation

def knn_self(X_train, X_test, y_train, k=5):
    """Implementation of an k-nearest neighbors classifier. 
    Returning predicted classes of samples to be predicted (X_test).

    Args:
        X_train (pd.DataFrame): Training data
        y_train (pd.Series): Target values of training data
        X_test (pd.DataFrame): Test data
        k (int, optional): Number of nearest neighbors considered for the classification. Defaults to 5.
    """

    pred_list = []                                            # Defining empty list to store the final predictions
    
    X_train = np.array(X_train)                               # Change format from dataframe to numpy array, since the self implemented... 
    X_test = np.array(X_test)                                 # ...distance function requires a numpy array as input format.
    y_train = np.array(y_train)

    for xp in X_test:                                         # Iterate through every test instance in order to predict a label 
        
        dist_list = []                                        # Defining empty list to temporarily store the calculated distance for a test...
                                                              # ...instance to all the single trainings instances. 
        for ind, xt in enumerate(X_train):                    # Iterate through every trainings instance to calculate the distance from the test instance. 
             
            dist = distance_self(xp, xt)                      # Calculate the distance for a test instance to a training instances
            dist_list.append((dist, ind))                     # Store the calculated distance and the index of the instance as tuple in list 'dist_list'
                
        list_sorted = sorted(dist_list)[:k]                   # Sort dist_list after distance and keep only the first k instances
            
        y_ind = [i[1] for i in list_sorted]                   # Get indexes from tuple of the k closest instances 
        y_labels = [y_train[i] for i in y_ind]                # Get labels from training labels for the collected indexes...
                                                              # ...returns a list 'y_labels' with the labels for the k closest training instances.
        y_pred = max(y_labels, key=y_labels.count)            # Get the most common label form the y_labels list --> Prediction for the test instance. 
            
        pred_list.append(y_pred)                              # Store the prediction in the pred_list. And start the for loop again for new test instance.
    
    return pred_list                                          # Return a list with all the calculated predicitons for the test instances. 

In [ ]:
# Get predictions for X_test with the knn_self
y_prediction = knn_self(X_train, X_test, y_train)

In [ ]:
# Evaluate the results of knn_self
confusion_matrix(y_test, y_prediction)

## Comparison with sklearn KNN implementation

That will be interesting! Check out how your implementation performs in comparison to the one of sklearn!
You can check the confusion matrix and the accuracy score of both algorithms.
If you want, you can check which algorithm is faster!

In [ ]:
# Test vs. sklearn implementation
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_knn = knn.predict(X_test)

In [ ]:
confusion_matrix(y_test, y_knn)